In [13]:
# pip install elasticsearch==7.10.1
# !pip install morfeusz2

In [1]:
!pip search elasticsearch

ERROR: XMLRPC request failed [code: -32500]
RuntimeError: PyPI's XMLRPC API is currently disabled due to unmanageable load and will be deprecated in the near future. See https://status.python.org/ for more information.


In [3]:
import re
import tarfile
import numpy as np
import glob
# import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from morfeusz2 import Morfeusz

In [4]:
import datetime
import logging
logger = logging.getLogger("elasticsearch")
logger.setLevel(logging.ERROR)
from elasticsearch import Elasticsearch
import os
import pandas as pd
import sys

In [5]:
es = Elasticsearch("http://localhost:9200")
resp = es.info()

In [6]:
resp

{'name': 'hypetraintommy',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': '5e2k1_kCR36mjX0pdK52rw',
 'version': {'number': '7.10.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '1c34507e66d7db1211f66f3513706fdf548736aa',
  'build_date': '2020-12-05T01:00:33.671820Z',
  'build_snapshot': False,
  'lucene_version': '8.7.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [7]:
try:
    es.indices.delete('polish_bills')
except:
    print("Index not created yet.")

Index not created yet.


/tmp/ipykernel_39577/437438416.py:2: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.delete('polish_bills')


In [8]:
index_body={
    "settings": {
        "index":{
            'analysis': {
                "analyzer": {
                    "polish_bills_analyzer":{
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": ["lowercase", "synonym", "morfologik_stem"]
                    }
                },
                "filter": {
                    "synonym":{
                        "type": "synonym",
                        "tokenizer": "standard",
                        "synonyms": ["kpk, kodeks postępowania karnego",
                                   "kpc, kodeks postępowania cywilnego",
                                   "kk, kodeks karny",
                                   "kc, kodeks cywilny"]
                    }
                }
            }
        }
    },
    "mappings": {
        "_doc": {
            "properties": {
                "text": {
                    "type": "text",
                    "analyzer": "polish_bills_analyzer"
                },
                "name": {"type": "text"}
            }
        }
    }
}

In [9]:
es.indices.create(index='polish_bills', include_type_name=True, body=index_body)

/tmp/ipykernel_39577/3175745924.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index='polish_bills', include_type_name=True, body=index_body)
/home/hypetraintommy/kubus/nlp2/kuba-nlp2/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: [types removal] Using include_type_name in create index requests is deprecated. The parameter will be removed in the next major version.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'polish_bills'}

In [11]:
os.listdir()[1].split()[0]

'2000_704.txt'

In [12]:
bills_list = []
for file in glob.glob("*.txt"):
    bills_list.append(file)
    with open(os.getcwd() + "/" + file, 'r', encoding='utf-8') as f:
        bill = f.read()
        year = file.split("_")[0]
        doc = {'file': year, 'text': bill}
        es.index(index='polish_bills', id=file, body=doc)

/tmp/ipykernel_39577/2864190266.py:8: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index='polish_bills', id=file, body=doc)


## Determine the number of legislative acts containing the word ustawa (in any form).

In [13]:
search_task6 = es.search(index='polish_bills', body = {
    "query": {
        "match": {
            "text": "ustawa"
        }
    }
}
         )
print("Legislative acts containing the word ustawa: {}".format(search_task6["hits"]["total"]["value"]))

/tmp/ipykernel_39577/1632681095.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  search_task6 = es.search(index='polish_bills', body = {


Legislative acts containing the word ustawa: 1178


## Determine the number of occurrences of the word ustawa by searching for this particular form, including the other inflectional forms.

In [31]:
search_task7 = es.search(index='polish_bills', request_timeout=100, body = {
    "query": {
        "match": {
            "text": "ustawa"
        }
    },
    "highlight": {
        "fields": {
            "text": {
                "number_of_fragments": 10000,
                "fragment_size": 1
            }
        }
    },
    "size": 10000
})

/tmp/ipykernel_39577/820971398.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  search_task7 = es.search(index='polish_bills', request_timeout=100, body = {


In [33]:
hits = 0
for hit in search_task7['hits']['hits']:
    hits += len(hit['highlight']['text'])
print("Number of occurences of the word ustawa and the other inflectional forms: {}".format(hits))

Number of occurences of the word ustawa and the other inflectional forms: 24934


## Determine the number of occurrences of the word ustaw by searching for this particular form, including the other inflectional forms.

In [34]:
search_task8 = es.search(index='polish_bills', request_timeout=100, body = {
    "query": {
        "match": {
            "text": "ustaw"
        }
    },
    "highlight": {
        "fields": {
            "text": {
                "number_of_fragments": 10000,
                "fragment_size": 1
            }
        }
    },
    "size": 5000
})

/tmp/ipykernel_39577/3239431428.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  search_task8 = es.search(index='polish_bills', request_timeout=100, body = {


In [35]:
print("Number of occurences of the word ustaw and the other inflectional forms: {}".format(hits))

Number of occurences of the word ustaw and the other inflectional forms: 24934


## Determine the number of legislative acts containing the words kodeks postępowania cywilnego in the specified order, but in any inflection form.

In [37]:
search_task9 = es.search(index='polish_bills', request_timeout=100, body = {
    "query": {
        "match": {
            "text": "kodeks postępowania cywilnego"
        }
    },
    "highlight": {
        "fields": {
            "text": {
                "number_of_fragments": 10000,
                "fragment_size": 1
            }
        }
    },
    "size": 5000
})

/tmp/ipykernel_39577/2777925528.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  search_task9 = es.search(index='polish_bills', request_timeout=100, body = {


In [40]:
print("Number of occurences of the words kodeks postępowania cywilnego and the other inflectional forms: {}".format(search_task9['hits']['total']['value']))

Number of occurences of the words kodeks postępowania cywilnego and the other inflectional forms: 99


## Determine the number of legislative acts containing the words wchodzi w życie (in any form) allowing for up to 2 additional words in the searched phrase.

In [41]:
search_task10 = es.search(index='polish_bills', request_timeout=100, body = {
    "query": {
        "match_phrase": {
            "text": {
                "query": "wchodzi w życie",
                "slop": 2
            }
        }
    }
})

/tmp/ipykernel_39577/871189906.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  search_task10 = es.search(index='polish_bills', request_timeout=100, body = {


In [42]:
print("Number of legislative acts containing the words wchodzi w życie (up to 2 additional words): {}".format(search_task10['hits']['total']['value']))

Number of legislative acts containing the words wchodzi w życie (up to 2 additional words): 1174


## Determine the 10 documents that are the most relevant for the phrase konstytucja.

In [59]:
search_task10 = es.search(index='polish_bills', request_timeout=100, body = {
    "query": {
        "match": {
            "text": {
                "query": "konstytucja"
            }
        }
    },
    "highlight": {
        "fields": {
            "text": {
                "number_of_fragments": 3
            }
        }
    },
    "size": 10
})
print("10 most relevant documents:")

10 most relevant documents:


/tmp/ipykernel_39577/2085503983.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  search_task10 = es.search(index='polish_bills', request_timeout=100, body = {


In [61]:
for hit, result in enumerate(search_task10["hits"]["hits"]):
    print("{}. -> bill no {}".format(hit+1, result['_id'][:-4]))

1. -> bill no 1997_629
2. -> bill no 2000_443
3. -> bill no 1997_604
4. -> bill no 1996_350
5. -> bill no 1997_642
6. -> bill no 2001_23
7. -> bill no 1996_199
8. -> bill no 1999_688
9. -> bill no 2001_1082
10. -> bill no 1997_681


## Print the excerpts containing the word konstytucja (up to three excerpts per document) from the previous task.

In [72]:
for hit in search_task10["hits"]["hits"]:
    print("{}:".format(hit["_id"][:-4]))
    for number, excerpt in enumerate(hit["highlight"]["text"]):
        print("{}: {}\n".format(number+1, excerpt))

1997_629:
1: o zmianie ustawy konstytucyjnej o trybie przygotowania
           i uchwalenia <em>Konstytucji</em> Rzeczypospolitej

2: W ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie przygotowania i 
uchwalenia <em>Konstytucji</em>

3: Do zgłoszenia projektu <em>Konstytucji</em> załącza się wykaz 
                obywateli popierających zgłoszenie

2000_443:
1: umowy międzynarodowej i nie wypełnia przesłanek określonych w art. 89
     ust. 1 lub art. 90 <em>Konstytucji</em>

2: międzynarodowej lub załącznika nie
     wypełnia przesłanek określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>

3: co do zasadności wyboru
  trybu ratyfikacji umowy międzynarodowej, o którym mowa w art. 89 ust. 2
  <em>Konstytucji</em>

1997_604:
1: Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności celów partii 
   politycznej z <em>Konstytucją</em>

2: Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności z <em>Konstytucją</em>
   celów lub działalności

3: Ciężar udow